1. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk sorszámozott listát a szálláshelyek adatairól!&nbsp;</span> 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
Select  row_number() over(
            PARTITION BY tipus
) as 'sorszám',
        *
from szallashely

2. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk lekérdezést, amely a foglalások adatait jeleníti meg!&nbsp;</span> 

a. A lista tartalmazzon két új oszlopot, amelyek az adott ügyfél legrövidebb, illetve leghosszabb foglalásának hosszát (a foglalt napok számát) mutatják meg! 

b. Az oszlopokat nevezzük el értelemszerűen!

In [ ]:
select  *,
        MAX(DATEDIFF(DAY, METTOL, MEDDIG)) over (PARTITION by UGYFEL_FK) as 'Leghosszabb foglalás',
        MIN(DATEDIFF(DAY, METTOL, MEDDIG)) over (PARTITION by UGYFEL_FK) as 'Legrövidebb foglalás'
from Foglalas

```
3. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 
```

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

In [ ]:
select  UGYFEL_FK, METTOL, MEDDIG,
        (FELNOTT_SZAM + GYERMEK_SZAM) as 'Össz fő',
        SUM((FELNOTT_SZAM + GYERMEK_SZAM)) over (PARTITION by UGYFEL_FK order by FOGLALAS_PK
        rows between 3 preceding and 1 preceding) as 'foglalasok össz fő'
from foglalas

4. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát arról, hogy az egyes szálláshelyeken évente hány foglalás történt!&nbsp;</span> 

a. A lista jelenítse meg a szálláshely azonosítóját, nevét, az évet, és a foglalások számát!

b. Egy új oszlopban jelenítsük meg a szálláshely eddigi éves foglalásainak számát is (az aktuálisat is beleértve). 

c. Az évnél a METTOL dátumot vegyük figyelembe! 

d. A lista ne tartalmazzon duplikált sorokat!

In [ ]:
SELECT distinct szh.SZALLAS_ID,YEAR(f.METTOL), count(*) over(partition by szh.SZALLAS_ID, YEAR(f.METTOL)),    
                -- eddigi foglalásk száma
                COUNT(*) over(partition by szh.szallas_id, year(f.METTOL)
                                order by f.mettol
                                range between unbounded preceding AND current ROW)
FROM Szallashely szh join szoba sz on szh.SZALLAS_ID=sz.SZALLAS_FK
                    join Foglalas f on sz.SZOBA_ID=f.SZOBA_FK

5. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsük listát a szálláshelyekről! A lista tartalmazza a szálláshely azonosítóját, nevét, és a csillagok számát csillagszám szerint csökkenő, azon belül szállásnév szerint növekvő sorrendben!&nbsp;</span> 

a. Vegyünk fel két új oszlopot, amely a sorrend szerint előző szálláshely nevét, illetve azonosítóját is megjeleníti!

In [ ]:
SELECT  szh.SZALLAS_ID,szh.SZALLAS_NEV,szh.CSILLAGOK_SZAMA,
        LAG(szh.SZALLAS_ID) over(order by szh.CSILLAGOK_SZAMA DESC,szh.szallas_nev ASC),
        LAG(szh.SZALLAS_NEV) over(order by szh.CSILLAGOK_SZAMA DESC,szh.szallas_nev ASC)
from szallashely szh
order by szh.CSILLAGOK_SZAMA DESC,szh.szallas_nev ASC

6. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon termékeket, amelyekből az átlagosnál kevesebb van raktáron!&nbsp;</span> 

a. Csak a termék kódja és megnevezése jelenjen meg!

In [ ]:
SELECT TERMEKKOD, MEGYS
From Termek
WHERE KESZLET < (
    Select avg(KESZLET)
    from Termek
)

7. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon ügyfeleket, akik még nem fizettek bankkártyával!&nbsp;</span> 

a. Csak az ügyfelek neve jelenjen meg!

In [ ]:
-- akiknek a kódja nincs benne a bankkártyával fiztők kódjaink halmazában
-- nem létezik olan rendelés, ahol az adott ügyfélkóddal foglalatak
-- és bakkártyával fizettek
Select nev
from Ugyfel u
where not exists(
    select *
    from Rendeles r
    where u.[LOGIN] = r.[LOGIN]
          and r.FIZ_MOD = 'Bankkártya'  
)

8. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Mennyibe kerül a második legnagyobb listaárú termék?</span>

In [9]:
select top 1 listaar
from Termek
where listaar < (
    select top 1 listaar
    from Termek
    ORDER by listaar DESC
)
order by listaar DESC


(1 row affected)

Total execution time: 00:00:00.019

listaar
39000


9. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik raktár(ak)ban van az a termék, amelyből a legnagyobb készlet van?&nbsp;</span> 

a. Csak a raktár(ak) neve jelenjen meg!

In [11]:
select r.RAKTAR_NEV
from Raktar r join termek t on r.RAKTAR_KOD = t.RAKTAR_KOD
where t.KESZLET = ALL(
    select MAX(t2.KESZLET)
    from termek t2
)

(2 rows affected)

Total execution time: 00:00:00.018

RAKTAR_NEV
Veresegyházi
Báthory


10. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára nagyobb, mint bármelyik olyan terméké, amelynek nevében az Ajándék szó szerepel?&nbsp;</span> 

a. A termékek kódját és nevét is jelenítsük meg!

In [7]:
select megnevezes
from Termek
where listaar > ALL(
    select listaar
    from Termek
    where charindex('ajándék', megnevezes) > 0
)


(122 rows affected)

Total execution time: 00:00:00.209

megnevezes
félfamentes rajzlap A4 (10 lap)
Öntapadó A/4
Fénymásoló papír DataCopy
Fénymásoló papír GetCopy
Fénymásoló papír Qconnect
Színes másoló a/4 80g
Fehér másoló A/3
Fehér másoló A/4
Beírókönyv A4
Határidő napló A/6


11. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik hónapokban adtak le az ügyfelek több rendelést, mint januárban?&nbsp;</span> 

a. Elég a hónapok sorszámait megjeleníteni. 

b. A listát rendezzük a hónapok száma szerint növekvő sorrendbe!

In [22]:
select MONTH(rend_datum)
from rendeles
group by MONTH(rend_datum)
having count(*) > all(
    select COUNT(*)
    from Rendeles
    where MONTH(rend_datum) = 1
)
order by MONTH(rend_datum)


(5 rows affected)

Total execution time: 00:00:00.033

(No column name)
3
7
8
10
12


12. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára az átlagostól max. 20%-kal tér el?&nbsp;</span> 

a. A termék kódját és nevét is jelenítsük meg!

In [ ]:
select termekkod, MEGNEVEZES
from Termek
where LISTAAR <=(
    select AVG(listaar)*1.2
    from Termek
) 
and LISTAAR >= (
    select AVG(listaar)*0.8
    from Termek
)

13. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát (sorszámonként) az egyes rendelések teljes összegéről!&nbsp;</span> 

a. Szűrjünk azon tételekre, ahol ez az érték nagyobb, mint Viktor Éva rendeléseinek teljes összege! 

b. A teljes összeg oszlopot nevezzük el értelemszerűen!

In [11]:
select  rt.SORSZAM as 'Sorszám',
        sum(egysegar*mennyiseg) as 'Teljes összeg' 
from Rendeles_tetel rt  join Rendeles r on rt.SORSZAM = r.SORSZAM
group by rt.SORSZAM
HAVING sum(egysegar*mennyiseg) > (
    select sum(egysegar*mennyiseg)
    from Rendeles_tetel rt  join Rendeles r on rt.SORSZAM = r.SORSZAM
                            join Ugyfel u on r.[LOGIN] = u.[LOGIN]
    where u.NEV = 'Viktor Éva'
)


Warning: Null value is eliminated by an aggregate or other SET operation.

(3205 rows affected)

Total execution time: 00:00:00.217

Sorszám,Teljes összeg
7,267994
21,535712
45,276762
53,306770
71,797340
72,213556
94,450222
107,211980
115,252810
130,242745


14. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azokat a termékkategóriákat, amelyből 2017 januárban nem történt rendelés!&nbsp;</span> 

a. Elég a kategóriák nevét megjeleníteni!

15. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon rendelési tételeket, amelynek értéke nagyobb, mint a legdrágább termék listaára azon termékek közül, amelyek szülő kategóriája a ruha!</span>

Készítsünk listát, amely megjeleníti a vendégek adatait!

- Egy új oszlopban számoljuk ki a vendég életkorát (években)
- Egy másik új oszlopban határozzuk meg, hogy születési dátum szerint növekvő rendezésnél mennyi az adott ügyfél. az előtte lévő 2 ügyfél és az utána lévő 2 ügyfél átlagos életkora! Az oszlopot nevezzük el értelemszerűen!

Egészítsük ki a megkezdett lekérdezést, amely listázza azon vendégek azonosítóját és nevét, akik már legalább egyszer foglaltak, és MINDEN ESETBEN összesen két fő számára (felnőtt + gyermek szám összege)! 

 a. A lista ne tartalmazzon ismétlődő sorokat!